In [ ]:
# Model trained in Keras/TF 2.0
# Steps to convert .h5 to .pb format to be able to load model in TF 1.xx cpp code

https://github.com/tensorflow/tensorflow/issues/29253#issuecomment-530782763

"I was able to generate a frozen tensorflow model from a TF 2.0 model. First step is to save the weights of the tf.keras model with model.save_weights('xxx.h5').

With a conda environment or another computer with TF 1.xx installed (I had 1.14), you generate an untrained tf.keras model identical to the model you want to freeze and load the weights with model.load_weights('xxx.h5').

At this point you can save the full model in a h5 file with model.save(...) and then freeze it."

In [1]:
import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Lambda, Flatten, Dense, Activation
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K

In [ ]:
# Do this step in TF2.0 environment
# load model trained in tf2-keras
#model = load_model("model_unet_checkpoint.h5")
#model.save_weights('result_w.h5')

In [2]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_CHANNELS = 3

In [3]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

s = Lambda(lambda x: (x - 0.0) / 255.0) (inputs)

c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', 
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (s)
c1 = BatchNormalization()(c1)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c1)
c1 = BatchNormalization()(c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (p1)
c2 = BatchNormalization()(c2)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c2)
c2 = BatchNormalization()(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (p2)
c3 = BatchNormalization()(c3)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c3)
c3 = BatchNormalization()(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (p3)
c4 = BatchNormalization()(c4)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c4)
c4 = BatchNormalization()(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (p4)
c5 = BatchNormalization()(c5)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c5)
cbn5 = BatchNormalization() (c5)

temp_p4 = MaxPooling2D(pool_size=(2, 2)) (cbn5)
temp_c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (temp_p4)
temp_c5 = BatchNormalization()(temp_c5)
temp_c5 = Dropout(0.3) (temp_c5)
temp_c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (temp_c5)

print(temp_c5.shape)
gap_p4 = GlobalAveragePooling2D() (temp_c5)
fc1_in = Flatten()(gap_p4)
fc1 = Dense(256*2, kernel_regularizer=tf.keras.regularizers.l2(l=0.001))(fc1_in)
fc1_relu = Activation("relu")(fc1)
fc2_in = Dropout(0.35)(fc1_relu)
fc2 = Dense(256*2, kernel_regularizer=tf.keras.regularizers.l2(l=0.001))(fc2_in)
fc2_relu = Activation("relu")(fc2)


fc3_in = Dropout(0.4)(fc2_relu)
fc3 = Dense(256*2,  kernel_regularizer=tf.keras.regularizers.l2(l=0.001))(fc3_in)
fc3_relu = Activation("relu")(fc3)

fc4_in = Dropout(0.4)(fc3_relu)

regression_output_predictions = Dense(4, name="regression_output")(fc4_in)

class_output_predictions = Dense(4, activation='softmax', name="class_output")(fc4_in)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same',
                     kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (cbn5)
u6 = concatenate([u6, c4])
c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (u6)
c6 = BatchNormalization()(c6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c6)
c6 = BatchNormalization()(c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', 
                     kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (u7)
c7 = BatchNormalization()(c7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c7)
c7 = BatchNormalization()(c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same',
                     kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (u8)
c8 = BatchNormalization()(c8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c8)
c8 = BatchNormalization()(c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same',
                     kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (u9)
c9 = BatchNormalization()(c9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001)) (c9)
c9 = BatchNormalization()(c9)

segmentation_output_map = Conv2D(3, (1, 1), activation='sigmoid', name='segmentation_output') (c9)

(?, 7, 7, 512)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
model = Model(inputs=[inputs], outputs=[segmentation_output_map, regression_output_predictions, class_output_predictions])

In [6]:
model.load_weights('result_w.h5')

In [7]:
model.save('trained_model.h5')

"I can now convert the trained model to a frozen pb file with python freeze_model.py --h5 trained_model.h5 --pb trained_model.pb"